In [1]:
from langgraph.graph import StateGraph,START,END
from langsmith import traceable
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver
from psycopg_pool import AsyncConnectionPool
from psycopg.rows import dict_row
from pydantic import BaseModel
from typing import Annotated,List,TypedDict
from langchain_core.messages import BaseMessage,HumanMessage
import operator
from pgvector.psycopg import register_vector_async

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
DATABASE_URL=os.getenv("DATABASE_URL")
pool=AsyncConnectionPool(DATABASE_URL)

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedder = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    api_key=os.getenv("GOOGLE_API_KEY")
)

In [5]:
async def init_db():
    async with pool.connection() as conn:
        await register_vector_async(conn)
    saver=AsyncPostgresSaver(pool)
    await saver.setup()
    return saver



In [6]:
saver = await init_db()


c:\Users\nasim\Desktop\JoluMate\server\venv\Lib\site-packages\psycopg_pool\pool_async.py:163: RuntimeWarning: opening the async pool AsyncConnectionPool in the constructor is deprecated and will not be supported anymore in a future release. Please use `await pool.open()`, or use the pool as context manager using: `async with AsyncConnectionPool(...) as pool: `...
  warnings.warn(


ProgrammingError: vector type not found in the database

In [29]:
class ChatState(TypedDict):
    messages:Annotated[List[BaseMessage],operator.add]

In [30]:
def chat_node(state:ChatState):
    message=state['messages']
    model=ChatGoogleGenerativeAI(model='gemini-2.5-flash',api_key=os.getenv("GOOGLE_API_KEY"))
    response=model.invoke(message)
    return {'messages':[response]}

In [ ]:
graph=StateGraph(ChatState)

graph.add_node('chat_node',chat_node)
graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)
chatbot=graph.compile(
    async_mode=True,
    checkpointer=AsyncPostgresSaver
)

In [38]:
initial_state={
    'messages':[HumanMessage(content="What is the weather now in domkal")]
}
response=chatbot.invoke(initial_state)
print(response)

{'messages': [HumanMessage(content='What is the weather now in domkal', additional_kwargs={}, response_metadata={}), AIMessage(content='I cannot provide real-time, live weather updates. My information is not current.\n\nTo get the most up-to-date weather for Domkal, I recommend checking a reliable weather website or app and searching for "weather in Domkal" or "Domkal weather now."', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b1220-6fa6-79f1-9c1e-bcc1b00ff745-0', usage_metadata={'input_tokens': 9, 'output_tokens': 314, 'total_tokens': 323, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 257}})]}


In [39]:
for msg in response["messages"]:
    print(type(msg), msg.content)


<class 'langchain_core.messages.human.HumanMessage'> What is the weather now in domkal
<class 'langchain_core.messages.ai.AIMessage'> I cannot provide real-time, live weather updates. My information is not current.

To get the most up-to-date weather for Domkal, I recommend checking a reliable weather website or app and searching for "weather in Domkal" or "Domkal weather now."


In [ ]:
import